In [1]:
%matplotlib inline
import numpy as np
from pycocotools.coco import COCO
import skimage.io as io
import matplotlib.pyplot as plt
import theano.tensor as T
import theano
import lasagne
import lasagne.nonlinearities as nonlin
import pylab
import pickle
import re
import os
import imageio
import time
from IPython.display import display, clear_output
import PIL

from net.vgg16 import make_network_from_file
from net.process import preprocess, resize_img

--- Logging error ---
Traceback (most recent call last):
  File "E:\WinPython-64bit-3.4.4.1\python-3.4.4.amd64\lib\logging\__init__.py", line 978, in emit
    msg = self.format(record)
  File "E:\WinPython-64bit-3.4.4.1\python-3.4.4.amd64\lib\logging\__init__.py", line 828, in format
    return fmt.format(record)
  File "E:\WinPython-64bit-3.4.4.1\python-3.4.4.amd64\lib\logging\__init__.py", line 573, in format
    record.exc_text = self.formatException(record.exc_info)
  File "E:\WinPython-64bit-3.4.4.1\python-3.4.4.amd64\lib\logging\__init__.py", line 523, in formatException
    traceback.print_exception(ei[0], ei[1], tb, None, sio)
  File "E:\WinPython-64bit-3.4.4.1\python-3.4.4.amd64\lib\traceback.py", line 169, in print_exception
    for line in _format_exception_iter(etype, value, tb, limit, chain):
  File "E:\WinPython-64bit-3.4.4.1\python-3.4.4.amd64\lib\traceback.py", line 146, in _format_exception_iter
    for value, tb in values:
  File "E:\WinPython-64bit-3.4.4.1\python-3.4

In [2]:
dataDir='annotations_trainval2017'
dataType='val2017'
annFile='{}/annotations/captions_{}.json'.format(dataDir, dataType)

coco=COCO(annFile)

loading annotations into memory...
Done (t=0.18s)
creating index...
index created!


In [3]:
def dict_from_anns(anns):
    return [(str(v[1]['image_id']), v[1]['caption']) for v in anns.items()]

# img2vec

In [4]:
from net.imgs import *
      
# %time imgs_process('annotations_trainval2017/img') # Wall time: 4h 43min 38s

In [5]:
imgs = imgs_load(dataDir)
display(len(imgs))

4991

# w2v

In [6]:
from net.word2vec import *

w2v = load_w2v(5*10**5)

E:\WinPython-64bit-3.4.4.1\python-3.4.4.amd64\lib\site-packages\gensim\utils.py:862: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
ERROR: 

Traceback (most recent call last):
  File "E:\WinPython-64bit-3.4.4.1\python-3.4.4.amd64\lib\site-packages\IPython\core\interactiveshell.py", line 2885, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-6-531d091b2409>", line 3, in <module>
    w2v = load_w2v(5*10**5)
  File "E:\HSE_deeplearning\net\word2vec.py", line 37, in load_w2v
    'net/GoogleNews-vectors-negative300.bin', binary=True, limit=limit)
  File "E:\WinPython-64bit-3.4.4.1\python-3.4.4.amd64\lib\site-packages\gensim\models\keyedvectors.py", line 242, in load_word2vec_format
    weights = fromstring(fin.read(binary_len), dtype=REAL)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "E:\WinPython-64bit-3.4.4.1\python-3.4.4.amd64\lib\site-packages\IPython\core\interactiveshell.py", line 1827, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '

Internal Python error in the inspect module.
Below is the traceback from this internal error.


KeyboardInterrupt


  File "E:\WinPython-64bit-3.4.4.1\python-3.4.4.amd64\lib\site-packages\IPython\core\ultratb.py", line 300, in wrapped
    return f(*args, **kwargs)
  File "E:\WinPython-64bit-3.4.4.1\python-3.4.4.amd64\lib\site-packages\IPython\core\ultratb.py", line 345, in _fixed_getinnerframes
    records = fix_frame_records_filenames(inspect.getinnerframes(etb, context))


# texts

In [7]:
from net.sents_process import *
from net.x_process import *

Traceback (most recent call last):
  File "E:\WinPython-64bit-3.4.4.1\python-3.4.4.amd64\lib\site-packages\IPython\core\interactiveshell.py", line 2885, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-7-7b69c2614809>", line 2, in <module>
    from net.x_process import *
ImportError: No module named 'net.x_process'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "E:\WinPython-64bit-3.4.4.1\python-3.4.4.amd64\lib\site-packages\IPython\core\interactiveshell.py", line 1827, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'ImportError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "E:\WinPython-64bit-3.4.4.1\python-3.4.4.amd64\lib\site-packages\IPython\core\ultratb.py", line 1118, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_conte

ERROR: Internal Python error in the inspect module.
Below is the traceback from this internal error.



KeyboardInterrupt



Unfortunately, your original traceback can not be constructed.



ImportError: No module named 'net.x_process'

In [ ]:
danno = sents_load(dataDir)

In [ ]:
captions = [text[1]['caption'] for text in coco.anns.items()]
display(captions[:5])

plt.hist(list(map(lambda x: len(x.split(' ')), captions)),bins=100)
t = np.argmax(list(map(lambda x: len(x.split(' ')), captions)))
display(captions[t])

# neural networks

In [ ]:
from net.train import *

# CNN

Generate X, y

In [ ]:
H = 300
W = 14

In [ ]:
train_size = 23300

    
    
X, y = [], []
for id, data in imgs.items():
    while id[0] == '0':
        id = id[1:]
    try:
        for ls in danno[id]:
            vects = []
            for elem in ls:
                if not isinstance(elem, str):
                    vects.append(elem)
            vects = procces_x(vects)
            X.append(vects)
            y.append(data[1])
    except:
        pass

print(len(X))

X_train, y_train, X_val, y_val = X[:train_size], y[:train_size], X[train_size:], y[train_size:]

# new version

In [ ]:
def build_cnn(file_name=None):
    net = NeuralNet(must_have=[
            'input_shape', 
            'learning_rate', 
            'train_fun', 
            'loss_fun', 
            'loss_fun_det', 
            'predict_fun_det'])
    
    input_shape = [None, H, W]
    
    input_X = T.tensor3("input X", dtype='float32')
    target_y = T.matrix("target Y", dtype='float32')

    
    net['inp'] = lasagne.layers.InputLayer(input_shape, input_var=input_X)
    
    net['max'] = lasagne.layers.GlobalPoolLayer(net['inp'], pool_function=theano.tensor.max)
    net['min'] = lasagne.layers.GlobalPoolLayer(net['inp'], pool_function=theano.tensor.min)
    net['mean'] = lasagne.layers.GlobalPoolLayer(net['inp'], pool_function=theano.tensor.mean)
    
    net['con_2'] = lasagne.layers.Conv1DLayer(net['inp'], num_filters=64, filter_size=2,nonlinearity=None)
    net['con_3'] = lasagne.layers.Conv1DLayer(net['inp'], num_filters=64, filter_size=3,nonlinearity=None)
    net['con_4'] = lasagne.layers.Conv1DLayer(net['inp'], num_filters=64, filter_size=4,nonlinearity=None)
    
    net['gmax_2'] = lasagne.layers.GlobalPoolLayer(net['con_2'], pool_function=T.max)
    net['gmax_3'] = lasagne.layers.GlobalPoolLayer(net['con_3'], pool_function=T.max)
    net['gmax_4'] = lasagne.layers.GlobalPoolLayer(net['con_4'], pool_function=T.max)
    
    net['merge'] = lasagne.layers.ConcatLayer((net['max'], net['min'], net['mean'], 
                                               net['gmax_2'], net['gmax_3'], net['gmax_4']))
    
    net['dens_1'] = lasagne.layers.DenseLayer(net['merge'], num_units=500, nonlinearity=nonlin.elu)
    net['batch_1'] =  lasagne.layers.batch_norm(net['dens_1'])
    net['drop_1'] = lasagne.layers.DropoutLayer(net['batch_1'], p=0.5)
    
    net['dens_2'] = lasagne.layers.DenseLayer(net['drop_1'], num_units=500, nonlinearity=nonlin.elu)
    net['batch_2'] =  lasagne.layers.batch_norm(net['dens_2'])
    net['drop_2'] = lasagne.layers.DropoutLayer(net['batch_2'], p=0.5)
    
    net['last'] = lasagne.layers.DenseLayer(net['drop_2'], num_units=4096)
    
    
    y_predicted = lasagne.layers.get_output(net['last'])
    y_predicted_det = lasagne.layers.get_output(net['last'], deterministic=True)

    all_weights = lasagne.layers.get_all_params(net['last'], trainable=True)
    
    learning_rate = theano.shared(lasagne.utils.floatX(0.001))
    loss = lasagne.objectives.squared_error(target_y, y_predicted).mean()
    loss_det = lasagne.objectives.squared_error(target_y, y_predicted_det).mean()
    
    # loss = loss + lasagne.regularization.regularize_layer_params(net['last'], lasagne.regularization.l2) * 0.01
    updates = lasagne.updates.adam(loss, all_weights, learning_rate=learning_rate)
    
    train_fun = theano.function([input_X, target_y], loss, updates=updates, allow_input_downcast=True)
    loss_fun = theano.function([input_X, target_y], loss, allow_input_downcast=True)
    loss_fun_det = theano.function([input_X, target_y], loss_det, allow_input_downcast=True)
    predict_fun_det = theano.function([input_X], y_predicted_det, allow_input_downcast=True)
    
    if file_name:
        load_net(net['last'], file_name, dataDir)
    
    net.input_shape = input_shape
    net.learning_rate = learning_rate
    net.train_fun = train_fun
    net.loss_fun = loss_fun
    net.loss_fun_det = loss_fun_det
    net.predict_fun_det = predict_fun_det
    
    return net.check()

In [ ]:
net = build_cnn('new_cnn2.3v_2.net')

# Analysis

In [ ]:
def text2vec(text):
    words = sent2words(text)
    vecs = []
    for word in words:
        try:
            vecs.append(w2v[word])
        except:
            pass
    return procces_x(vecs)

text2vec('dog run').shape

In [ ]:
def show_img(img):
    img = PIL.Image.fromarray(img)
    img = img.resize((200,200), PIL.Image.ANTIALIAS)
    plt.imshow(img)
    plt.show()
    
def show_img_by_id(img_id, dataDir=dataDir):
    show_img(imageio.imread(dataDir + '/img/' + img_id + '.jpg'))

In [ ]:
def get_imgs_by_vec(vec, imgs=imgs):
    return [(id, np.linalg.norm(vec - point[1])) for id, point in imgs.items()]

def show_t2i(text, count=3, net=net):
    vec = net.predict_fun_det([text2vec(text)])[0]
    good_imgs = get_imgs_by_vec(vec)
    good_imgs.sort(key=lambda x: x[1])
    good_imgs = good_imgs[:count]
    display(good_imgs)
    for id, _ in good_imgs:
        print(id)
        show_img_by_id(id)

In [ ]:
show_t2i('baby with mother')

In [ ]:
def process_cap(mod=100):
    result = []
    for i, text in enumerate(captions)
        result.append(net.predict_fun_det([text2vec(text)])[0])
        if i % mod == 0:
            print('Processed {}'.format(i))
    return result

procesed_captions = process_cap()

In [ ]:
def get_texts_by_vec(vec, texts=captions, procesed_texts=procesed_captions):
    result = []
    for text, point in zip(texts, procesed_texts):
        dis = np.linalg.norm(vec - point)
        result.append((text, dis))
    result.sort(key=lambda x: x[1])
    return result

def show_i2t(img_id, count=3, imgs=imgs):
    show_img_by_id(img_id)
    vec = imgs[img_id][1]
    good_text = get_texts_by_vec(vec)[:count]
    print(*good_text, sep='\n', end='\n\n')

In [ ]:
show_i2t('000000565597', count=5)
show_i2t('000000564280', count=5)
show_i2t('000000554291', count=5)

In [ ]:
def get_texts_by_text(text, net=net, texts=captions, procesed_texts=procesed_captions):
    vec = net.predict_fun_det([text2vec(text)])[0]
    result = []
    for text, point in zip(texts, procesed_texts):
        dis = np.linalg.norm(vec - point)
        result.append((text, dis))
    result.sort(key=lambda x: x[1])
    return result

def show_t2t(text, count=3):
    print(text)
    good_text = get_texts_by_text(text)[:count]
    print(*good_text, sep='\n', end='\n\n')

In [ ]:
show_t2t('A big dog is running')
show_t2t('Green apple')
show_t2t('donut on the table')

# Video

In [ ]:
import process

def frames_load(dir_name):
    frames = {}
    for file_name in os.listdir(dir_name + '/video_data'):
        if '.' in file_name and file_name.split('.')[1] == 'data':
            with open('{}/video_data/{}'.format(dir_name, file_name), 'rb') as fl:
                data = pickle.load(fl)
                frames.update(data)
    return frames

frames = frames_load(dataDir)

In [ ]:
imgs_v = {k: v['fc7'] for k, v in frames.items()}

In [ ]:
video_name = 'Africa.mp4'
vd = imageio.get_reader(video_name)

In [ ]:
import scipy.interpolate as interpolate
from pandas import ewma

def show_video_t2i(text, count=3, net=net):
    vec = net.predict_fun_det([text2vec(text)])[0]
    good_frames = get_imgs_by_vec(vec, imgs=imgs_v)
    good_frames.sort(key=lambda x: x[1])
    good_frames = good_frames[:count]
    display(good_frames)
    for id, _ in good_frames:
        print(id)
        show_img(vd.get_data(id))
        
def show_video_t2graph(text, net=net):
    vec = net.predict_fun_det([text2vec(text)])[0]
    good_frames = get_imgs_by_vec(vec, imgs=imgs_v)
    good_frames.sort(key=lambda x: x[1])
    
    mx = good_frames[50][1]
    good_frames.sort(key=lambda x: x[0])
    
    x, y = np.transpose(good_frames)
    
    pic = np.array(list(map(lambda t: 1.0 if t < mx else 0.0, y)))
    wind = 20
    res = []
    res_x = []
    for i in range(len(pic) - wind):
        s = 0
        for j in range(wind):
            s += pic[i + j]
        res.append(s)
        res_x.append(x[i] / 25 / 60)
    
    plt.plot(res_x, res)
    plt.show()
    
    return
    x, y = np.transpose(good_frames)
    y = ewma(y, span=3)
    y = y - np.min(y)
    y = np.log(y)
    y = ewma(y, span=2)
    plt.plot(y)
    plt.show()

In [ ]:
show_video_t2graph('a couple of people that are playing in a field')

In [ ]:
show_video_t2i('tiger tiger tiger')

In [ ]:
def link_from_time(time, id='8kBGjNI2bwA'):
    return "https://youtu.be/{}?t={}".format(id, time)

In [ ]:
show_i2t(10, imgs=imgs_v)